In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch  
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune import ExperimentAnalysis


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Neural Net Class and Training Functions
Define Class and functions

In [2]:
def data_loaders():
   # Import data
    dir_X = '/Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/ev_adoption_ml/Data/df_X_county.csv'
    dir_y = '/Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/ev_adoption_ml/Data/df_y_county.csv'
    
    X = pd.read_csv(dir_X)
    #X['constant'] = 1
    y = pd.read_csv(dir_y)

    # check if any nan values
    nan_row_X = X[X.isna().any(axis=1)]
    #print(nan_row_X)
    nan_row_y = y[y.isna().any(axis=1)]
    #print(nan_row_y)

    X = X.to_numpy()
    y = y.to_numpy()

    # only predict PHEV
    #y = y[:,0]

    # split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    

    # standardize X
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def train_model(nn_model, data_loaded, opt, batch_size=32):
    
    '''
    Trains neural network model on X_train, y_train data.
    
    Returns
    ----------
    nn_model: torch.nn.Module
        trained neural network model
    '''
    # convert to tensors (for Pytorch)
    X_train, X_eval, y_train, y_eval = data_loaded
    X_train_tensor = torch.tensor(X_train)
    y_train_tensor = torch.tensor(y_train)
    X_test_tensor = torch.tensor(X_eval)
    y_test_tensor = torch.tensor(y_eval)
    
    # train with (mini-batch) SGD; initialize optimizer
    n_samples, n_features = X_train_tensor.shape
    # initialize mse loss function
    mse_loss = torch.nn.MSELoss()
    nn_model.train()  # put model in train mode
    
    # loop through data in batches
    for batch_start in range(0, n_samples, batch_size):
    # reset gradients to zero
        opt.zero_grad()
        # form batch
        X_batch = X_train_tensor[batch_start:batch_start+batch_size]
        y_batch = y_train_tensor[batch_start:batch_start+batch_size]
        X_batch_test = X_test_tensor[batch_start:batch_start+batch_size]
        y_batch_test = y_test_tensor[batch_start:batch_start+batch_size]
        # pass batch through neural net to get prediction
        y_pred = nn_model(X_batch.float())
        y_pred = y_pred.unsqueeze(1)
        y_pred_test = nn_model(X_batch_test.float())
        y_pred_test = y_pred_test.unsqueeze(1)
        # compute MSE loss
        loss = mse_loss(y_pred, y_batch[:, None].float())
        loss_test = mse_loss(y_pred_test, y_batch_test[:, None].float())
        # back-propagate loss
        loss.backward()
        # update model parameters based on backpropogated gradients - clip values to avoid exploding gradients
        torch.nn.utils.clip_grad_value_(nn_model.parameters(), clip_value=1.5)
        opt.step()
        
        #print(f"Mean Train MSE: {epoch_loss}")
        
    return loss, loss_test

def evaluate_model(nn_model, X_eval, y_eval, batch_size=32):
    '''
    Evaluates trained neural network model on X_eval, y_eval data.

    Parameters
    ----------
    nn_model: torch.nn.Module
        trained neural network model
    X_eval: np.array
        matrix of training data features
    y_eval: np.array
        vector of training data labels
    batch_size: int
        batch size to looping over dataset to generate predictions

    Returns
    ----------
    mse: float
        MSE of trained model on X_eval, y_eval data
    '''
    # initialize mse loss function
    mse_loss = torch.nn.MSELoss()
    # convert to tensors (for Pytorch)
    X_eval_tensor = torch.tensor(X_eval)
    y_eval_tensor = torch.tensor(y_eval)
    n_samples = X_eval_tensor.shape[0]
    nn_model.eval() # put in eval mode
    # loop over data and generate predictions
    preds = []
    for batch_start in range(0, n_samples, batch_size):
        # form batch
        X_batch = X_eval_tensor[batch_start:batch_start+batch_size]
        y_batch = y_eval_tensor[batch_start:batch_start+batch_size]
        with torch.no_grad():  # no need to compute gradients during evaluation
            # pass batch through neural net to get prediction
            y_pred = nn_model(X_batch.float())
            y_pred = y_pred.unsqueeze(1)
            preds.append(y_pred)
    # compute MSE across all samples
    all_preds = torch.cat(preds)
    loss = mse_loss(all_preds, y_eval_tensor[:, None].float()).item()
    return loss

def train_and_validate(config):

    '''Parameters
    ----------
    X_train: np.array
        matrix of training data features
    y_train: np.array
        vector of training data labels
    max_iter: int
        maximum number of iterations to train for
    batch_size: int
        batch size to use when training w/ SGD
    '''
    # intialize neural network
    data_loaded = data_loaders()
    X_train, X_eval, y_train, y_eval = data_loaded
    print(y_eval)
    n_samples, n_features = X_train.shape
    nn_model = NN_configureable(n_features, config["n_hidden_dim"], config["n_layers"])

    opt = torch.optim.SGD(nn_model.parameters(), lr=config["lr"],  momentum=0.9)
    #opt = torch.optim.SGD(nn_model.parameters(), lr=config["lr"])
    batch_size = config["batch_size"]
    max_iter = config["train_iterations"]

    # Start the training.
    for it in range(max_iter):
        # save losses across all batches
        train_epoch_loss, test_epoch_loss = train_model(nn_model, data_loaded, opt, batch_size)
        valid_epoch_loss = evaluate_model(nn_model, X_eval, y_eval, batch_size)

        with tune.checkpoint_dir(it) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, 'checkpoint')
                torch.save((nn_model.state_dict(), opt.state_dict()), path)
        tune.report(
            loss=valid_epoch_loss)

class NN(nn.Module):
    '''
    Class for fully connected neural net.
    '''
    def __init__(self, input_dim, hidden_dim):
        '''
        Parameters
        ----------
        input_dim: int
            input dimension (i.e., # of features in each example passed to the network)
        hidden_dim: int
            number of nodes in hidden layer
        '''
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.layers = nn.Sequential(
            # Network has a single hidden layer
            # Apply ReLU activation in between the hidden layer and output node
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.layers(x)
        return x


class NN_configureable(nn.Module):
    '''
    Class for fully connected neural net.
    '''
    def __init__(self, input_dim, hidden_dim, hidden_layers):
        '''
        Parameters
        ----------
        input_dim: int
            input dimension (i.e., # of features in each example passed to the network)
        hidden_dim: int
            number of nodes in hidden layer
        '''
        super().__init__()
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        #self.layers = nn.ModuleDict()
        self.layers = nn.ModuleDict()
        
        # Define input layer
        self.layers["input"] = nn.Linear(in_features = input_dim, out_features = hidden_dim)
        # Define hidden layers
        for i in range(self.hidden_layers):
            self.layers[f"hidden_{i}"] = nn.Linear(in_features = hidden_dim, out_features = hidden_dim)
        # Define output layer
        self.layers["output"] = nn.Linear(in_features = hidden_dim, out_features = 1)

    def forward(self, x):
        x = self.layers["input"](x)
        for i in range(self.hidden_layers):
            x = F.relu(self.layers[f"hidden_{i}"](x))

        return self.layers["output"](x)        
        

# Search Function for Ray Tune
Hyperparameter search



# Import Data
Import combined data

In [3]:
def main():   
    
    # Search Function for Ray Tune - Hyperparameter search
     
    #X = pd.read_csv('./Data/df_X_county.csv')
    #print(X.head)
     
    X_train, X_eval, y_train, y_eval = data_loaders()
    n_samples, n_features = X_train.shape

    # Define the parameter search configuration.
    config = {
        "n_layers": 
            #tune.sample_from(lambda _: 2 ** np.random.randint(1, 5)),
            tune.grid_search([1, 2, 3, 4, 8]),
        "n_hidden_dim": 
            #tune.sample_from(lambda _: 2 ** np.random.randint(4, 8)),
            tune.grid_search([2, 4, 8, n_features]),
        "lr": tune.loguniform(1e-5, 1e-3),
        "batch_size": tune.choice([32, 64]),
        "train_iterations": tune.choice([50, 100])
    }

    max_num_iter = 50
    grace_period = 1
    # Number of Ray Tune random search experiments to run.
    num_samples = 20
    
    # Schduler to stop bad performing trails.
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t = max_num_iter,
        grace_period = grace_period,
        reduction_factor = 2 
    )

    # Reporter to show on command line/output window
    reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])


    # Start Ray Tune search
    result = tune.run(
        train_and_validate,
        resources_per_trial = {"cpu": 2, "gpu": 0},
        config = config,
        num_samples = num_samples,
        scheduler = scheduler,
        local_dir = '../outputs/raytune_result',
        keep_checkpoints_num = 1,
        checkpoint_score_attr = 'min-validation_loss',
        progress_reporter = reporter)

    # Extract the best trial run from the search.
    best_trial = result.get_best_trial('loss', 'min', 'last')
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    
    #result.dataframe().csv("df_raytune_search.csv")

    '''
    plt.xlabel('Iteration Step')
    plt.ylabel('Test Error')
    plt.title("Model 1 - Hidden Layer - ReLU")
    plt.legend()
    plt.figure(figsize=(20,12))
    plt.show()
'''

In [4]:
if __name__ == '__main__':
    main()

2023-05-12 08:58:30,197	INFO worker.py:1625 -- Started a local Ray instance.
2023-05-12 08:58:31,026	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


== Status ==
Current time: 2023-05-12 08:58:31 (running for 00:00:00.07)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-12_08-58-31
Number of trials: 16/400 (15 PENDING, 1 RUNNING)
+--------------------------------+----------+-----------------+--------------+-------------+----------------+------------+--------------------+
| Trial name                     | status   | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |
|--------------------------------+----------+-----------------+--------------+-------------+----------------+------------+--------------------|
| train_and_validate_b2159_00000 | RUNNING  | 127.0.0.1:92410 |           64 | 0.000204454 |              

Trial name,date,done,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_and_validate_b2159_00000,2023-05-12_08-58-35,True,Dons-MacBook-Pro.local,50,281781,127.0.0.1,92410,True,2.58995,0.0522859,2.58995,1683896315,50,b2159_00000
train_and_validate_b2159_00001,2023-05-12_08-58-34,True,Dons-MacBook-Pro.local,1,526933,127.0.0.1,92411,True,0.077493,0.077493,0.077493,1683896314,1,b2159_00001
train_and_validate_b2159_00002,2023-05-12_08-58-34,True,Dons-MacBook-Pro.local,1,667369,127.0.0.1,92412,True,0.056042,0.056042,0.056042,1683896314,1,b2159_00002
train_and_validate_b2159_00003,2023-05-12_08-58-40,True,Dons-MacBook-Pro.local,50,247403,127.0.0.1,92413,True,5.69688,0.100965,5.69688,1683896320,50,b2159_00003
train_and_validate_b2159_00004,2023-05-12_08-58-34,True,Dons-MacBook-Pro.local,1,667410,127.0.0.1,92411,True,0.04687,0.04687,0.04687,1683896314,1,b2159_00004
train_and_validate_b2159_00005,2023-05-12_08-58-39,True,Dons-MacBook-Pro.local,50,268030,127.0.0.1,92412,True,4.53559,0.141661,4.53559,1683896319,50,b2159_00005
train_and_validate_b2159_00006,2023-05-12_08-58-35,True,Dons-MacBook-Pro.local,2,359673,127.0.0.1,92411,True,0.16378,0.0848987,0.16378,1683896315,2,b2159_00006
train_and_validate_b2159_00007,2023-05-12_08-58-41,True,Dons-MacBook-Pro.local,50,176628,127.0.0.1,92411,True,6.67271,0.113749,6.67271,1683896321,50,b2159_00007
train_and_validate_b2159_00008,2023-05-12_08-58-35,True,Dons-MacBook-Pro.local,1,666654,127.0.0.1,92410,True,0.0651419,0.0651419,0.0651419,1683896315,1,b2159_00008
train_and_validate_b2159_00009,2023-05-12_08-58-35,True,Dons-MacBook-Pro.local,1,667806,127.0.0.1,92410,True,0.0934551,0.0934551,0.0934551,1683896315,1,b2159_00009


2023-05-12 08:58:32,820	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/train_iterations']
2023-05-12 08:58:32,880	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/t

== Status ==
Current time: 2023-05-12 08:58:36 (running for 00:00:05.07)
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 32.000: -292048.96875 | Iter 16.000: -320866.0625 | Iter 8.000: -272172.703125 | Iter 4.000: -290280.09375 | Iter 2.000: -327393.796875 | Iter 1.000: -557621.8125
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-12_08-58-31
Number of trials: 27/400 (16 PENDING, 4 RUNNING, 7 TERMINATED)
+--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+--------+----------------------+
| Trial name                     | status     | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |   loss |   training_iteration |
|--------------------------------+------------+-----------------+--------------+-------------+----------------+---

...

(train_and_validate pid=92412) [[ 13.] [repeated 20x across cluster]
(train_and_validate pid=92412)  [122.] [repeated 40x across cluster]
(train_and_validate pid=92412)  [ 10.]] [repeated 60x across cluster]
(train_and_validate pid=92412)  ... [repeated 20x across cluster]


2023-05-12 09:00:10,911	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/train_iterations']
2023-05-12 09:00:10,951	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/t

== Status ==
Current time: 2023-05-12 09:00:11 (running for 00:01:40.25)
Using AsyncHyperBand: num_stopped=392
Bracket: Iter 32.000: -123290.51171875 | Iter 16.000: -166058.859375 | Iter 8.000: -223828.0 | Iter 4.000: -276886.1875 | Iter 2.000: -347205.90625 | Iter 1.000: -666740.28125
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-12_08-58-31
Number of trials: 400/400 (4 PENDING, 4 RUNNING, 392 TERMINATED)
+--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+----------+----------------------+
| Trial name                     | status     | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |     loss |   training_iteration |
|--------------------------------+------------+-----------------+--------------+-------------+-------------

2023-05-12 09:00:11,405	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/train_iterations']
2023-05-12 09:00:11,406	ERROR checkpoint_manager.py:361 -- Result dict has no key: validation_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag', 'config/n_layers', 'config/n_hidden_dim', 'config/lr', 'config/batch_size', 'config/t

== Status ==
Current time: 2023-05-12 09:00:14 (running for 00:01:43.57)
Using AsyncHyperBand: num_stopped=400
Bracket: Iter 32.000: -123290.51171875 | Iter 16.000: -163803.25 | Iter 8.000: -223828.0 | Iter 4.000: -277506.59375 | Iter 2.000: -347205.90625 | Iter 1.000: -666791.90625
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Users/donokoye/Documents/Spring_23/ML_1.C51/Final_Project/outputs/raytune_result/train_and_validate_2023-05-12_08-58-31
Number of trials: 400/400 (400 TERMINATED)
+--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+--------------------+----------+----------------------+
| Trial name                     | status     | loc             |   batch_size |          lr |   n_hidden_dim |   n_layers |   train_iterations |     loss |   training_iteration |
|--------------------------------+------------+-----------------+--------------+-------------+----------------+------------+----------